In [1]:
# 04 - modelo 2.ipynb

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Cargar el conjunto de datos
file_path = 'dataset_02052023.xlsx'
df = pd.read_excel(file_path)

# Preprocesamiento
df.drop(['Timestamp', 'Num'], axis=1, inplace=True, errors='ignore')
if df['grip_lost'].dtype == 'object':
    df['grip_lost'] = df['grip_lost'].astype(int)
df.dropna(subset=['Robot_ProtectiveStop'], inplace=True)
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

# Dividir los datos en características y variable objetivo
X = df.drop('Robot_ProtectiveStop', axis=1)
y = df['Robot_ProtectiveStop']

# Reducir el tamaño del dataset para pruebas (10% del total)
X, _, y, _ = train_test_split(X, y, train_size=0.1, stratify=y, random_state=42)

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Construir un pipeline para el modelo Random Forest
rf_pipe = Pipeline([
    ('rf', RandomForestClassifier())
])

# Definir la grilla de hiperparámetros para GridSearchCV
param_grid_rf = {
    'rf__n_estimators': [100, 200],
    'rf__max_depth': [10, 20]
}

# Realizar búsqueda de hiperparámetros para Random Forest
rf_grid = GridSearchCV(rf_pipe, param_grid_rf, cv=3, error_score='raise')
rf_grid.fit(X_train, y_train)

# Obtener los mejores hiperparámetros y el mejor modelo Random Forest
best_params_rf = rf_grid.best_params_
best_model_rf = rf_grid.best_estimator_

# Evaluar el modelo en el conjunto de prueba
y_pred_rf = best_model_rf.predict(X_test)

# Imprimir métricas de evaluación
print("Random Forest:")
print(classification_report(y_test, y_pred_rf))


Random Forest:
              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98       212
         1.0       1.00      0.11      0.20         9

    accuracy                           0.96       221
   macro avg       0.98      0.56      0.59       221
weighted avg       0.97      0.96      0.95       221

